In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime
import json
import logging
import pandas as pd
from dateutil import parser
import re



: 

In [2]:
# Send a GET request to the URL
url = "https://www.regattanetwork.com/clubmgmt/applet_regatta_results.php?regatta_id=26198"
page = requests.get(url)

In [61]:
img_url = soup.body.img['src']
img_url = "https://www.regattanetwork.com/clubmgmt" + img_url[1:]
img_url


'https://www.regattanetwork.com/clubmgmt/club_uploads/204/SSS Logo.png'

In [1]:
import json
import pandas as pd
import requests
from dateutil import parser
from bs4 import BeautifulSoup

# Send a GET request to the URL
url = "https://www.regattanetwork.com/clubmgmt/applet_regatta_results.php?regatta_id=26198"
page = requests.get(url)

# Initialize an empty dictionary to store JSON objects for each iteration
payload = {}

# Get the HTML content of the page
soup = BeautifulSoup(page.content, 'html.parser')

# Get title
title_text = soup.head.title.text

# Find event description
h4_tags = soup.find_all('h4')
event_description_tag = h4_tags[0].text.strip()

# Split the event description based on '\r\n'
event_description_parts = event_description_tag.split('\r\n')

# Split the second part of the event description based on ' | ' to get location and date
location_and_date = event_description_parts[1].split(' | ')

# Extract event name, location, and date
event_name = event_description_parts[0]
location = location_and_date[0]
date = location_and_date[1]

parsed_date = parser.parse(date)

# Download image
img_url = soup.body.img['src']
img_url = "https://www.regattanetwork.com/clubmgmt" + img_url[1:]
response = requests.get(img_url)

# Check if the request was successful
if response.status_code == 200:
    # Save the image to a file
    with open("event_image.jpg", "wb") as f:
        f.write(response.content)
else:
    print("Failed to download image")

# Add title, event name, location, date, and image URL to the payload dictionary
payload['title'] = title_text
payload['event_name'] = event_name
payload['location'] = location
payload['parsed_date'] = parsed_date.isoformat()
payload['image_url'] = img_url

# Find all tables containing race results
race_results = soup.find_all('div', class_='scoring-table-wrapper')

# Find all h2 tags containing fleet information
h2_tags = soup.find_all('h2')

# Iterate over each fleet and its corresponding race results
for index, tag in enumerate(h2_tags):
    # Extract fleet name and description
    fleet_name = tag.a['name']
    fleet_description = tag.find('font', size='3').text.strip()

    # Initialize the JSON object for this fleet
    fleet_data = {
        'fleet_name': fleet_name,
        'fleet_description': fleet_description
    }

    # Find the corresponding table for this fleet
    table = race_results[index].find('table', class_='scoring-table')
    
    # Check if table exists
    if table:
        # Read the HTML table into a DataFrame, skipping the first row
        df = pd.read_html(str(table), flavor='bs4', header=0)[0]
        
        # Drop specific columns if they exist
        columns_to_drop = ['Unnamed: 6', 'Unnamed: 11']
        df = df.drop(columns=columns_to_drop, errors='ignore')

        # Convert DataFrame to JSON-compatible format
        fleet_data['data'] = df.to_dict(orient='records')
    else:
        fleet_data['data'] = []  # Empty list if table does not exist

    # Add fleet data to the payload dictionary with index as key
    payload[index] = fleet_data

# Convert the payload dictionary to a JSON string
json_data = json.dumps(payload, indent=4)

# Now, json_data contains the JSON string representing the payload dictionary



C:\Users\danie\AppData\Local\Temp\ipykernel_10436\4166025702.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\danie\AppData\Local\Temp\ipykernel_10436\4166025702.py:81: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4', header=0)[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_10436\4166025702.py:81: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal

In [2]:
import requests
from bs4 import BeautifulSoup
import argparse

def fetch_and_save_html(url):
    # Fetch HTML data from the URL
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    response.content



def main(start_year, end_year=None, num_pages=None):
    if end_year is None:
        end_year = datetime.datetime.now().year + 1
    all_data = []
    for year in range(start_year, end_year):
        base_url = f"https://www.regattanetwork.com/html/results.php?year={year}"
        regatta_links = extract_regatta_links(base_url)
        logging.info(f"Year {year}: Found {len(regatta_links)} regatta links")
        for link in regatta_links:
            scraped_data = fetch_and_save_html(link)
            if scraped_data:  # Ensure scraped_data is not empty
                all_data.append(scraped_data)
                if num_pages and len(all_data) >= num_pages:
                    break
    
    # Saving the compiled data into a JSON file

    with open('regattas_data.json', 'w') as json_file:
        json.dump(all_data, json_file, indent=4)
        logging.info("Successfully saved all scraped data to regattas_data.json")

if __name__ == "__main__": 
    parser = argparse.ArgumentParser(description='Scrape regatta data.')
    parser.add_argument('start_year', type=int, help='The start year for scraping')
    parser.add_argument('--end_year', type=int, help='The end year for scraping')
    parser.add_argument('--num_pages', type=int, help='The number of pages to scrape')
    
    args = parser.parse_args()
    main(args.start_year, args.end_year, args.num_pages)


{
    "title": "\r\n2024 Cherry Pie Regatta SBYA Event - Series Standing",
    "event_name": "2024 Cherry Pie Regatta SBYA Event",
    "location": "  Sarasota Sailing Squadron",
    "parsed_date": "2024-02-17T00:00:00",
    "image_url": "https://www.regattanetwork.com/clubmgmt/club_uploads/204/SSS Logo.png",
    "0": {
        "fleet_name": "Melges15",
        "fleet_description": "Series Standing - 4 races scored",
        "data": [
            {
                "Pos": NaN,
                "Sail": NaN,
                "Boat": NaN,
                "Rating": NaN,
                "Skipper": NaN,
                "Yacht Club": NaN,
                "1": NaN,
                "2": NaN,
                "3": NaN,
                "4": NaN,
                "Total": NaN,
                "Pos.1": NaN
            },
            {
                "Pos": 1.0,
                "Sail": 107.0,
                "Boat": NaN,
                "Rating": 0.0,
                "Skipper": "Jeff Eiber",
            

In [48]:
import json
import pandas as pd

# Initialize an empty dictionary to store JSON objects for each iteration
payload = {}

# Assuming race_results contains the soup object with all the tables
for index, tag in enumerate(h2_tags):
    # Initialize the JSON object for this iteration
    json_object = {}

    # Extract fleet name and description
    fleet_name = tag.a['name']
    fleet_description = tag.find('font', size='3').text.strip()

    # Add fleet name and description to the JSON object
    json_object['fleet_name'] = fleet_name
    json_object['fleet_description'] = fleet_description

    # Find the table inside the corresponding race_results element
    table = race_results[index].find('table', class_='scoring-table')
    
    # Check if table exists
    if table:
        # Use Pandas to read the HTML table into a DataFrame, skipping the first row
        df = pd.read_html(str(table), flavor='bs4')[0]

        # Convert DataFrame to a list of dictionaries
        df_json = df.to_dict(orient='records')

        # Add DataFrame to the JSON object
        json_object['data'] = df_json
    else:
        json_object['data'] = []  # Empty list if table does not exist

    # Add the JSON object to the payload dictionary with index as key
    payload[index] = json_object

# Convert the payload dictionary to a JSON string
json_data = json.dumps(payload, indent=4)

# Now, json_data contains the JSON string representing the payload dictionary


C:\Users\danie\AppData\Local\Temp\ipykernel_29480\2110407307.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4')[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\2110407307.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4')[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\2110407307.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4')[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\2110407307.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will 

In [49]:
df

,Pos,Sail,Boat,Rating,Skipper,Yacht Club,Corrected,Unnamed: 7,1,Unnamed: 9,Total,Pos.1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,777.0,Whole Lotta Rosie,18.0,Jon Dowd,Sarasota Sailing Squadron,00:47:30,NaN,1,NaN,1.0,1.0
2,2.0,7.0,Sunspot,9.0,Bill Johnsen,William Johnsen,00:49:05,NaN,2,NaN,2.0,2.0
3,3.0,12.0,Triple Threat,-33.0,John Novak,Sarasota Sailing Squadron,00:50:38,NaN,3,NaN,3.0,3.0
4,4.0,187.0,Krakian,24.0,Alan Staniforth,SSS/SYS,00:52:58,NaN,4,NaN,4.0,4.0
5,5.0,138.0,NaN,-15.0,NaN,Sss,NO TIME,NaN,6/DNC,NaN,6.0,5.0


In [8]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import argparse
import datetime
import json
import logging
import uuid
import csv
from jinja2 import Template, Environment
from pydantic import BaseModel, Field
from typing import Dict, Optional, Any
import psycopg2
import yaml
from io import StringIO
import pydantic

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


class DBTarget(BaseModel):
    type: str
    keepalives_idle:int
    connect_timeout:int
    retries:int
    user:str
    schema:str
    account:Optional[str]=None
    role:Optional[str]=None
    database:str
    password:Optional[str]=None
    port:int

DBTarget.update_forward_refs()

class DBTProfile(BaseModel):
    target: str
    outputs: Dict[str, DBTarget]

DBTProfile.update_forward_refs()

def env_var(variable_name:str, default:Optional[str]=None) -> Optional[str]:
    debug(f'Getting environment variable: {variable_name}')
    value = os.getenv(variable_name, default)

jinja_env = Environment()
jinja_env.filters['env_var'] = env_var


def get_profiles()->Dict[str,DBTProfile]:
    with open('profiles.yml') as f:
        template=jinja_env.from_string(f.read())

    profiles_string=template.render()
    obj:Dict[str,Any]=yaml.load(StringIO(profiles_string),Loader=yaml.SafeLoader)
    if 'config' in obj.keys():
        obj.pop('config')
    for k,v in obj.items():
        if 'defaults' in v['outputs']:
            v['outputs'].pop('defaults')
    
    profiles=pydantic.parse_obj_as(Dict[str,DBTProfile],obj)
    return profiles

def connect_to_postgres(db_target: DBTarget):
    try:
        conn = psycopg2.connect(
            dbname=db_target.database,
            user=db_target.user,
            password=db_target.password,
            host=db_target.account,
            port=db_target.port 
        )
        logging.info("Successfully connected to PostgreSQL")
        return conn
    except Exception as e:
        logging.exception("Error connecting to PostgreSQL")
        return None


profiles=get_profiles()
target = profiles['interactive'].outputs['dv']

test_db_connection(target)

c:\Users\danie\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_internal\_fields.py:201: UserWarning: Field name "schema" in "DBTarget" shadows an attribute in parent "BaseModel"
  warnings.warn(
C:\Users\danie\AppData\Local\Temp\ipykernel_25620\145354977.py:63: PydanticDeprecatedSince20: `parse_obj_as` is deprecated. Use `pydantic.TypeAdapter.validate_python` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  profiles=pydantic.parse_obj_as(Dict[str,DBTProfile],obj)
2024-07-30 15:10:19,671 - INFO - Successfully connected to PostgreSQL
2024-07-30 15:10:19,672 - INFO - Database connection test successful


In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import argparse
# from postgres_connection import get_postgres_connection
import logging
from datetime import datetime
import requests


logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')


def html_table_to_dataframe_with_links(html_content):

    logging.debug("Parsing HTML content")
    tables = pd.read_html(html_content)
    
    # Assuming the first table is the one you want
    df = tables[1]
    df = df.drop(index=0).reset_index(drop=True)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the table rows
    table = soup.find_all('table')[1]  # Assuming the second table is the one you want
    rows = table.find_all('tr')
    
    # Extract links and add them to the DataFrame
    links = []
    for row in rows:
        logging.debug("Processing rows")
        row_links = []
        cells = row.find_all('td')
        logging.debug(f"Found {len(cells)} cells in the row")
        for cell in cells:
            link = cell.find('a')
            if link:
                row_links.append(link.get('href'))
            else:
                row_links.append(None)
        if row_links:
            logging.debug(f"Adding links: {row_links}")
            links.append(row_links)
    
    # Convert the list of links to a DataFrame
    links_df = pd.DataFrame(links, columns=df.columns)
    
    # Rename the columns in the links DataFrame
    links_df.columns = ['Regatta Name', 'Host Club', 'Location', 'Regatta Date', 'Results']
    
    # Combine the original DataFrame with the links DataFrame
    combined_df = df.join(links_df, rsuffix='_link')
    return combined_df

def get_ingestioned_regatta_list(year):
    
    try:
        connection = get_postgres_connection('interactive','dv')
        logging.debug("Connection successful")
        # Execute SQL query
        query = f"SELECT * FROM ingestion_records WHERE condition = 'regatta_date' AND EXTRACT(YEAR FROM regatta_date) IN {year}"
        cursor = connection.cursor()
        cursor.execute(query)
        # Fetch all the rows from the cursor
        rows = cursor.fetchall()
        # Create a DataFrame from the fetched rows
        result_df = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
 
        # Close the cursor and connection
        cursor.close()
        connection.close()
        return result_df
    except Exception as e:
        raise Exception("Connection failed") from e

def extract_regatta_results_page(url):
    try:
        response = requests.get(url)
        logging.debug(f"Status code returned: {response.status_code}")
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        return response.content
    except requests.exceptions.RequestException as e:
        logging.error(f"Error extracting regatta links from: {url}. Error: {e}")


def main(year):
    combined_df = pd.DataFrame()
    for y in year:
        base_url = f"https://www.regattanetwork.com/html/results.php?year={y}"
        logging.debug(f"Scraping {y} regatta network data {base_url}")
        df_with_links = html_table_to_dataframe_with_links(extract_regatta_results_page(base_url))
        combined_df = pd.concat([combined_df, df_with_links], ignore_index=True)

        # ingested_list = get_ingestioned_regatta_list(year)
    return combined_df.head(20)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Process HTML file and extract table with links.')
    parser.add_argument('--year', nargs='+', type=int, help='List of years', default=[datetime.now().year])
    args = parser.parse_args()
    main(args.year)

usage: ipykernel_launcher.py [-h] [--year YEAR [YEAR ...]]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\danie\AppData\Roaming\jupyter\runtime\kernel-v2-8360Tr4L7JjjUvPs.json


SystemExit: 2

In [10]:
import json
from bs4 import BeautifulSoup

# Initialize a list to store JSON objects for each table
all_table_data = []


race_results = soup.find_all('div', class_='scoring-table-wrapper')

# Assuming race_results contains the soup object with all the tables
for table_wrapper in race_results:
    # Find the table inside the wrapper
    table = table_wrapper.find('table', class_='scoring-table')
    
    # Check if table exists
    if table:
        # Initialize a list to store JSON data for this table
        table_data = []

        # Iterate through each row in the table, skipping the first row which contains the headers
        for row in table.find_all('tr')[1:]:
            # Extract data from each column in the row
            columns = row.find_all('td')
            position = columns[0].text.strip()
            sail_number = columns[1].text.strip()
            skipper_name = columns[4].text.strip()
            total_score = columns[-2].text.strip()

            # Create a JSON object for this row and append it to the table data list
            row_data = {
                'Position': position,
                'Sail Number': sail_number,
                'Skipper Name': skipper_name,
                'Total Score': total_score
            }
            table_data.append(row_data)

        # Append the JSON data for this table to the list containing all tables' data
        all_table_data.append(table_data)
    else:
        print("No table found in the wrapper.")

# Convert the list of table data JSON objects to a JSON string
json_data = json.dumps(all_table_data, indent=4)

# Print the JSON string
print(json_data)


[
    [
        {
            "Position": "Pos",
            "Sail Number": "Sail",
            "Skipper Name": "Skipper",
            "Total Score": "Total"
        },
        {
            "Position": "1",
            "Sail Number": "107",
            "Skipper Name": "Jeff Eiber",
            "Total Score": "4"
        },
        {
            "Position": "2",
            "Sail Number": "107",
            "Skipper Name": "",
            "Total Score": "8"
        },
        {
            "Position": "3",
            "Sail Number": "011",
            "Skipper Name": "Eva McKee",
            "Total Score": "13"
        },
        {
            "Position": "4",
            "Sail Number": "013",
            "Skipper Name": "LANCE THOMAS",
            "Total Score": "15"
        },
        {
            "Position": "5",
            "Sail Number": "1",
            "Skipper Name": "Jeff Nicholas",
            "Total Score": "21"
        },
        {
            "Position": "6",
            

In [23]:
import pandas as pd

# Initialize an empty list to store DataFrames for each table
dfs = []

# Assuming race_results contains the soup object with all the tables
for table_wrapper in race_results:
    # Find the table inside the wrapper
    table = table_wrapper.find('table', class_='scoring-table')
    
    # Check if table exists
    if table:
        # Extract column headers
        headers = [header.text.strip() for header in table.find_all('td')]
        
        # Initialize a list to store data for this table
        data = []

        # Iterate through each row in the table, skipping the first row which contains the headers
        for row in table.find_all('tr')[1:]:
            # Extract data from each column in the row
            columns = row.find_all('td')
            row_data = [column.text.strip() for column in columns]

            # Append the data for this row to the table data list
            data.append(row_data)

        # Create a DataFrame for this table
        df = pd.DataFrame(data, columns=headers)
        
        # Append the DataFrame to the list of DataFrames
        dfs.append(df)
    else:
        print("No table found in the wrapper.")

# Now, dfs will contain a list of DataFrames, each representing a table from the HTML


ValueError: 126 columns passed, passed data had 14 columns

In [185]:
h4_tags = soup.find_all('h4')
event_description_tag = h4_tags[0].text.strip()

# Split the event description based on '\r\n'
event_description_parts = event_description_tag.split('\r\n')

# Split the second part of the event description based on ' | ' to get location and date
location_and_date = event_description_parts[1].split(' | ')

# Extract event name, location, and date
event_name = event_description_parts[0]
location = location_and_date[0]
date = location_and_date[1]

print("Event Name:", event_name)
print("Location:", location)
print("Date:", date)


Event Name: 2024 Cherry Pie Regatta SBYA Event
Location:   Sarasota Sailing Squadron
Date:         February 17, 2024


In [161]:
h2_tags = soup.find_all('h4')
for tag in h2_tags:
    print(tag)

<h4>2024 Cherry Pie Regatta SBYA Event<br/>
  Sarasota Sailing Squadron |         February 17, 2024        </h4>
<h4>
</h4>
<h4><font size="2">
        Information is provisional and subject to modification<br/>
<small>Regatta results last updated: Saturday, February 17, 2024 4:07:38 PM CDT<br/>Click on race number to view detailed race information.</small></font></h4>
<h4><font size="2">
        Information is provisional and subject to modification<br/>
<small>Regatta results last updated: Saturday, February 17, 2024 4:18:07 PM CDT<br/>Click on race number to view detailed race information.</small></font></h4>
<h4><font size="2">
        Information is provisional and subject to modification<br/>
<small>Regatta results last updated: Saturday, February 17, 2024 3:55:38 PM CDT<br/>Click on race number to view detailed race information.</small></font></h4>
<h4><font size="2">
        Information is provisional and subject to modification<br/>
<small>Regatta results last updated: Saturda

In [48]:
h2 = soup.find('h2')
print(h2)


<h2>
<a href="/clubmgmt/applet_regatta_results.php?regatta_id=26198&amp;limit_fleet=Melges15" name="Melges15"><font size="4">Melges15</font></a> <font size="4"><small><small>(<small>8</small> boats) <a href="#top">(top)<br/>
</a></small></small></font><font size="3">
  Series Standing - 4 races scored</font></h2>


In [36]:
soup

results_tables = soup.find_all('table', class_ = 'responsive scoring-table')



for tables in results_tables:
    print(tables.title)

None
None
None
None
None
None
None
None
None
None


In [27]:
soup.find_all('table')[3
]

<table bgcolor="#999999" class="responsive scoring-table">
<tr>
<!-- <td bgcolor="#999999"> -->
<thead>
<tr>
<td align="center" bgcolor="#6699CC" class="pos" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Pos</b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Sail  </b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Boat  </b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Rating</b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Skipper</b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Yacht Club</b></font></font></td>
<td align="center" bgcolor="#999999" valign="bottom"></td>
<td align="center" bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>  <a h

In [18]:
soup = BeautifulSoup(page.text, 'html.parser')

In [28]:
import pandas as pd
from bs4 import BeautifulSoup

def html_table_to_dataframe_with_links(file_path):
    # Read the HTML file
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Use pandas to read the HTML content and extract tables
    tables = pd.read_html(html_content)
    
    # Assuming the first table is the one you want
    df = tables[1]
    df = df.drop(index=0).reset_index(drop=True)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the table rows
    table = soup.find_all('table')[1]  # Assuming the second table is the one you want
    rows = table.find_all('tr')
    
    # Extract links and add them to the DataFrame
    links = []
    for row in rows:
        row_links = []
        cells = row.find_all('td')
        for cell in cells:
            link = cell.find('a')
            if link:
                row_links.append(link.get('href'))
            else:
                row_links.append(None)
        if row_links:
            links.append(row_links)
    
    # Convert the list of links to a DataFrame
    links_df = pd.DataFrame(links, columns=df.columns)
    
    # Combine the original DataFrame with the links DataFrame
    combined_df = df.join(links_df, rsuffix='_link')
    
    return combined_df

# Example usage
file_path = 'Results Archive from Past Events on Regatta Network.html'
df_with_links = html_table_to_dataframe_with_links(file_path)

C:\Users\danie\AppData\Local\Temp\ipykernel_25620\538080194.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html_content)
